## 데이터 로드

In [5]:
import tensorflow as tf

import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_df = pd.read_csv('cifar_trainset.csv')
test_df = pd.read_csv('cifar_testset.csv')

print(train_df.shape, test_df.shape)

(50000, 2) (10000, 2)


In [6]:
train_df.head()

,name,class
0,../dataset/cifar/train\0_frog.png,frog
1,../dataset/cifar/train\10000_automobile.png,automobile
2,../dataset/cifar/train\10001_frog.png,frog
3,../dataset/cifar/train\10002_frog.png,frog
4,../dataset/cifar/train\10003_ship.png,ship


## HyperParamter

In [26]:
num_epochs=10
batch_size=32

learning_rate = 0.001
dropout_rate = 0.7

input_shape = (32,32,3)
num_classes = 10

## 데이터 Generator

In [27]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255.,
)

########################
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    target_size = input_shape[:2],
    x_col='name', y_col = 'class',
    batch_size=batch_size
)

########################
valid_generator = test_datagen.flow_from_dataframe(
    test_df,
    target_size = input_shape[:2],
    x_col='name', y_col = 'class' ,
    batch_size = batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [28]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3,3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = (2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs = net, name = 'BasicCNN')
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


## Training

In [29]:
print(batch_size)
print(len(train_df) // 32)
print(len(train_generator)) # 반올림 차이

32
1562
1563


In [30]:
model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),
    epochs=1, # num_epochs,
    validation_data = valid_generator,
    validation_steps = len(valid_generator)
)

1563/1563 [==============================] - 621s 398ms/step - loss: 1.7550 - accuracy: 0.3589 - val_loss: 1.3737 - val_accuracy: 0.5151
